In [1]:
import torch.nn as nn

필터를 언제 사용하는가?
- 1*1 : 연산량 축소 (세밀한 특징 추출) : 눈, 코, 입 high-level
- 3*3 : 중간 특징 (얼굴 부분) mid-level
- 5*5 : 큰 특징(전체 얼굴) low level

왜 1*1 커널의 중요한가?
- 병렬 처리 (인셉션: 동시처리)

일반적인 CNN : Low level, mid-level, high-level
incepction : 병렬 처리 (인셉션 구조)

- 계산 효율성
  - 일반 5x5 합성곱 : 5x5x512x512 = 6,553,600 연산
  - 인셉션 (1x1 --> 5x5) 1x1x512x24 + 5x5x24x64 = 50,688 연산 >> 129배 빨라요

In [2]:
# InceptionModule 클래스를 정의함. nn.Module을 상속받아 PyTorch 모듈로 만듦.
class InceptionModule(nn.Module):
    # 인셉션 모듈을 초기화함. 입력 채널 수(input_planes)와 각 브랜치(branch)의 채널 수를 인수로 받음.
    def __init__(self, input_planes, n_channels1x1, n_channels3x3red, n_channels3x3, n_channels5x5red, n_channels5x5, pooling_planes):
        # 부모 클래스(nn.Module)의 생성자를 호출함.
        super(InceptionModule, self).__init__()

        # 첫 번째 브랜치: 1x1 합성곱만 수행함.
        # 빠른 처리, 세부 특징 추출
        self.block1 = nn.Sequential(
            # 1x1 커널 크기의 합성곱 레이어를 정의함.
            nn.Conv2d(input_planes, n_channels1x1, kernel_size=1),
            # 배치 정규화(BatchNorm)를 적용함.
            nn.BatchNorm2d(n_channels1x1),
            # ReLU 활성화 함수를 적용함.
            nn.ReLU(True),
        )

        # 두 번째 브랜치: 1x1 합성곱(채널 축소) -> 3x3 합성곱을 수행함.
        # 중간 크기 패턴 감지
        self.block2 = nn.Sequential(
            # 1x1 합성곱으로 채널을 축소함. (n_channels3x3red)
            nn.Conv2d(input_planes, n_channels3x3red, kernel_size=1),
            nn.BatchNorm2d(n_channels3x3red),
            nn.ReLU(True),
            # 3x3 합성곱을 적용함. padding=1로 출력 해상도를 유지함.
            nn.Conv2d(n_channels3x3red, n_channels3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_channels3x3),
            nn.ReLU(True),
        )

        # 세 번째 브랜치: 1x1 합성곱(채널 축소) -> 5x5와 동등한 3x3 합성곱 두 번을 수행함.(5*5 효과)
        # 큰 패턴 감
        self.block3 = nn.Sequential(
            # 1x1 합성곱으로 채널을 축소함. (n_channels5x5red)
            nn.Conv2d(input_planes, n_channels5x5red, kernel_size=1),
            nn.BatchNorm2d(n_channels5x5red),
            nn.ReLU(True),
            # 첫 번째 3x3 합성곱을 적용함.
            nn.Conv2d(n_channels5x5red, n_channels5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_channels5x5),
            nn.ReLU(True),
            # 두 번째 3x3 합성곱을 적용함. 5x5 합성곱과 유사한 효과를 냄.
            nn.Conv2d(n_channels5x5, n_channels5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_channels5x5),
            nn.ReLU(True),
        )

        # 네 번째 브랜치: 3x3 맥스 풀링 -> 1x1 합성곱(채널 축소)을 수행함.
        # 중요한 특징 보존
        self.block4 = nn.Sequential(
            # 3x3 맥스 풀링을 적용함. stride=1, padding=1로 해상도를 유지함.
            nn.MaxPool2d(3, stride=1, padding=1),
            # 1x1 합성곱으로 채널을 축소함. (pooling_planes)
            nn.Conv2d(input_planes, pooling_planes, kernel_size=1),
            nn.BatchNorm2d(pooling_planes),
            nn.ReLU(True),
        )

    # 데이터가 모듈을 통과하는 순서를 정의함.
    def forward(self, ip):
        # 각 브랜치(block)에 입력(ip)을 통과시켜 출력을 얻음.
        op1 = self.block1(ip)
        op2 = self.block2(ip)
        op3 = self.block3(ip)
        op4 = self.block4(ip)

        # 네 브랜치의 출력을 채널 차원(1번 차원)을 따라 하나로 병합(concatenate)하여 반환함.
        return torch.cat([op1,op2,op3,op4], 1)

In [3]:
# GoogLeNet 클래스를 정의함. nn.Module을 상속받아 전체 신경망 구조를 만듦.
class GoogLeNet(nn.Module):
    # 모델의 레이어들을 정의하여 초기화함.
    def __init__(self):
        # 부모 클래스(nn.Module)의 생성자를 호출했음.
        super(GoogLeNet, self).__init__()
        # 초기 특징 추출부(stem)를 정의함. 3x3 합성곱, 배치 정규화, ReLU로 구성됨.
        self.stem = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )

        # 첫 번째 Inception 모듈(im1)을 정의함. 입력 192 채널을 받음.
        # inception module 9개 연속 배치
        # InceptionModule(입력, 1*1, 3*3축소,3*3, 5*5축소, 5*5, pooling)
        self.im1 = InceptionModule(192, 64, 96, 128, 16, 32, 32)
        # 두 번째 Inception 모듈(im2)을 정의함. 입력 256 채널을 받음.
        self.im2 = InceptionModule(256, 128, 128, 192, 32, 96, 64)
        # im1 : 192 >> (64+128+32+32) = 256
        # im2 : 256 >> (128+192+96+64) = 480


        # 해상도 감소를 위한 맥스 풀링 레이어를 정의했음.
        self.max_pool = nn.MaxPool2d(3, stride=2, padding=1)

        # 세 번째에서 일곱 번째 Inception 모듈(im3 ~ im7)을 정의함.
        self.im3 = InceptionModule(480, 192, 96, 208, 16, 48, 64)
        self.im4 = InceptionModule(512, 160, 112, 224, 24, 64, 64)
        self.im5 = InceptionModule(512, 128, 128, 256, 24, 64, 64)
        self.im6 = InceptionModule(512, 112, 144, 288, 32, 64, 64)
        self.im7 = InceptionModule(528, 256, 160, 320, 32, 128, 128)

        # 여덟 번째와 아홉 번째 Inception 모듈(im8, im9)을 정의함.
        self.im8 = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.im9 = InceptionModule(832, 384, 192, 384, 48, 128, 128)

        # 최종 특징 추출을 위한 평균 풀링 레이어를 정의했음.
        self.average_pool = nn.AvgPool2d(7, stride=1)
        # 최종 분류를 위한 완전 연결 레이어(1000개 클래스)를 정의함.
        self.fc = nn.Linear(4096, 1000)

    # 데이터의 순전파 경로를 정의함.
    def forward(self, ip):
        # 입력(ip)을 stem 모듈에 통과시켜 op를 얻음.
        op = self.stem(ip)
        # im1 모듈을 통과시켜 op에 저장함. (이후 op 변수가 다시 사용됨)
        op = self.im1(op)
        # im2 모듈을 통과시켜 op에 덮어씀.
        op = self.im2(op)
        # maxpool 레이어를 사용하여 op를 처리함. (주의: self.max_pool 대신 maxpool 변수가 사용됨)
        op = self.maxpool(op)
        # a4 레이어를 통해 op를 처리함. (주의: init에 정의되지 않은 변수임)
        op = self.a4(op)
        # b4 레이어를 통해 op를 처리함. (주의: init에 정의되지 않은 변수임)
        op = self.b4(op)
        # c4 레이어를 통해 op를 처리함. (주의: init에 정의되지 않은 변수임)
        op = self.c4(op)
        # d4 레이어를 통해 op를 처리함. (주의: init에 정의되지 않은 변수임)
        op = self.d4(op)
        # e4 레이어를 통해 op를 처리함. (주의: init에 정의되지 않은 변수임)
        op = self.e4(op)
        # max_pool 레이어를 통과시켜 해상도를 줄였음.
        op = self.max_pool(op)
        # a5 레이어를 통해 op를 처리함. (주의: init에 정의되지 않은 변수임)
        op = self.a5(op)
        # b5 레이어를 통해 op를 처리함. (주의: init에 정의되지 않은 변수임)
        op = self.b5(op)
        # avgerage_pool 레이어를 통과시켰음. (주의: self.average_pool 대신 오타 변수가 사용됨)
        op = self.avgerage_pool(op)
        # 배치 차원을 제외하고 텐서를 평탄화(flatten)했음.
        op = op.view(op.size(0), -1)
        # 최종 완전 연결 레이어(fc)를 통과시켜 최종 출력을 얻음.
        op = self.fc(op)
        # 최종 결과를 반환함.
        return op

In [4]:
lenet = GoogLeNet()
lenet

GoogLeNet(
  (stem): Sequential(
    (0): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (im1): InceptionModule(
    (block1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (block2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (block3): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16,

In [5]:
# eos